# Sprawozdanie 11

**Grupa A3:**

inż. Michał Liss

inż. Marceli Sokólski

inż. Piotr Krzystanek

## Definicje funkcji

In [1]:
import paramiko
import os
import re
import requests
import json
import time

In [2]:
def run_in_master(command):
    ssh = paramiko.SSHClient()
    ssh.set_missing_host_key_policy(paramiko.AutoAddPolicy())
    ssh.connect("namenode", username="root", password="pass")
    ssh_stdin, ssh_stdout, ssh_stderr = ssh.exec_command(f"cd /app/ && . /env_var_path.sh && {command}")
    return (ssh_stdout.readlines(), ssh_stderr.readlines())

def merge_results(path):
    run_in_master(f"hdfs dfs -cat {path}/part-* | hdfs dfs -put - {path}/merged.txt")
    
def get_data_from_output_path(path):
    return f"{path}/merged.txt"

def print_hdfs_output(path):
    raw = run_in_master(f"hdfs dfs -cat {get_data_from_output_path(path)}")[0]
    print("\n".join(raw[0:11]))
    
def get_time(res, max_attempts: int = 6):
    def get_id(res):
        for line in res[1]:
            m = re.search('tracking URL: http://resourcemanager:8088/proxy/(.*)/', line)
            if m != None and m.group(1) != '':
                return m.group(1)
        return ''

    def get_time_from_data(data):
        sum = 0
        for attemp in data['attempts']:
            if attemp['completed'] is False:
                # print('spark history server has not updated (yet)')
                return -1
            sum = sum + attemp['duration']
        return sum
        
    id = get_id(res)
    if id == -1:
        return -1

    attempt = 0

    while attempt < max_attempts:
        attempt = attempt + 1
        response = requests.get(f'http://namenode:18080/api/v1/applications/{id}')
        if not response.ok:
            print('WARNING: application error')
            return -1
            
        data = json.loads(response.text)
        t = get_time_from_data(data)

        if t >= 0:
            return t
        time.sleep(5)
    print('WARNING: maximum attempts exceeded')
    return -1

runs = 10

# Covid

In [4]:
measurements_covid_sql = []
measurements_covid_df = []

## SQL

In [4]:
for r in range(runs):
    run_in_master(f"hdfs dfs -rm -r /spark-result/covid/sql")
    measurements_covid_sql.append(get_time(run_in_master("spark-submit --master yarn --deploy-mode cluster /data/master_volume/spark_scripts/covid_sql.py")))
    print(f"run {r} took {measurements_covid_sql[-1]}ms")
    
merge_results("/spark-result/covid/sql")
print_hdfs_output("/spark-result/covid/sql")

run 0 took 18062ms
run 1 took 17356ms
run 2 took 14562ms
run 3 took 14290ms
run 4 took 14166ms
run 5 took 13484ms
run 6 took 13527ms
run 7 took 13840ms
run 8 took 13995ms
run 9 took 13635ms
date,location,total_cases,new_cases,total_deaths,new_deaths,new_cases_per_million,average_new_cases_per_million

2020-01-03,Afghanistan,,0,,0,0,160.00584439903545

2020-01-04,Afghanistan,,0,,0,0,160.00584439903545

2020-01-05,Afghanistan,,0,,0,0,160.00584439903545

2020-01-06,Afghanistan,,0,,0,0,160.00584439903545

2020-01-07,Afghanistan,,0,,0,0,160.00584439903545

2020-01-08,Afghanistan,,0,,0,0,160.00584439903545

2020-01-09,Afghanistan,,0,,0,0,160.00584439903545

2020-01-10,Afghanistan,,0,,0,0,160.00584439903545

2020-01-11,Afghanistan,,0,,0,0,160.00584439903545

2020-01-12,Afghanistan,,0,,0,0,160.00584439903545



## DataFrame

In [5]:
for r in range(runs):
    run_in_master(f"hdfs dfs -rm -r /spark-result/covid/df")
    measurements_covid_df.append(get_time(run_in_master("spark-submit --master yarn --deploy-mode cluster /data/master_volume/spark_scripts/covid_df.py")))
    print(f"run {r} took {measurements_covid_df[-1]}ms")
    
merge_results("/spark-result/covid/df")
print_hdfs_output("/spark-result/covid/df")

run 0 took 16242ms
run 1 took 15807ms
run 2 took 15567ms
run 3 took 15292ms
run 4 took 15817ms
run 5 took 15751ms
run 6 took 15171ms
run 7 took 15048ms
run 8 took 15236ms
run 9 took 19541ms
date,location,total_cases,new_cases,total_deaths,new_deaths,new_cases_per_million,average_new_cases_per_million

2022-05-03,Curacao,42035,0,273,0,0,160.00584439903545

2022-05-04,Curacao,42330,295,274,1,1543,160.00584439903545

2022-05-05,Curacao,42330,0,274,0,0,160.00584439903545

2022-05-06,Curacao,42330,0,274,0,0,160.00584439903545

2022-05-07,Curacao,42330,0,274,0,0,160.00584439903545

2022-05-08,Curacao,42330,0,274,0,0,160.00584439903545

2022-05-09,Curacao,42330,0,274,0,0,160.00584439903545

2022-05-10,Curacao,42330,0,274,0,0,160.00584439903545

2022-05-11,Curacao,42674,344,275,1,1799,160.00584439903545

2022-05-12,Curacao,42674,0,275,0,0,160.00584439903545



# Steam

In [3]:
measurements_steam_sql = []
measurements_steam_df = []

## SQL

In [7]:
for r in range(runs):
    run_in_master(f"hdfs dfs -rm -r /spark-result/steam/sql")
    measurements_steam_sql.append(get_time(run_in_master("spark-submit --master yarn --deploy-mode cluster /data/master_volume/spark_scripts/steam_sql.py")))
    print(f"run {r} took {measurements_steam_sql[-1]}ms")
    
merge_results("/spark-result/steam/sql")
print_hdfs_output("/spark-result/steam/sql")

run 0 took 17226ms
run 1 took 17619ms
run 2 took 15960ms
run 3 took 15022ms
run 4 took 15969ms
run 5 took 16073ms
run 6 took 16086ms
run 7 took 15478ms
run 8 took 15842ms
run 9 took 14839ms
steam_appid,coming_soon,date,appid,name,positive,negative,owners,ccu

907680,false,"Aug 17, 2018",907680,Wwbit,710,566,"100,000 .. 200,000",0

823550,false,"Sep 18, 2018",823550,Booty Calls,255,294,"100,000 .. 200,000",134

639780,false,"Dec 7, 2017",639780,Deep Space Waifu: FLAT JUSTICE,1449,67,"50,000 .. 100,000",4

steam_appid,coming_soon,date,appid,name,positive,negative,owners,ccu

896890,false,"Dec 23, 2019",896890,VR Paradise - Steam Edition,138,50,"20,000 .. 50,000",11

726360,false,"May 22, 2020",726360,BOOBS SAGA: Prepare To Hentai Edition,367,103,"20,000 .. 50,000",0

723090,false,"Oct 24, 2017",723090,Meltys Quest,446,10,"20,000 .. 50,000",36

712790,false,"Oct 2, 2017",712790,Crimson Memories,51,21,"20,000 .. 50,000",0

825300,false,"Nov 22, 2018",825300,To Trust an Incubus,43,5,"0 .. 2

## DataFrame

In [4]:
for r in range(runs):
    run_in_master(f"hdfs dfs -rm -r /spark-result/steam/df")
    measurements_steam_df.append(get_time(run_in_master("spark-submit --master yarn --deploy-mode cluster /data/master_volume/spark_scripts/steam_df.py")))
    print(f"run {r} took {measurements_steam_df[-1]}ms")
    
merge_results("/spark-result/steam/df")
print_hdfs_output("/spark-result/steam/df")

run 0 took 18774ms
run 1 took 15032ms
run 2 took 15259ms
run 3 took 16165ms
run 4 took 14997ms
run 5 took 15087ms
run 6 took 14952ms
run 7 took 15014ms
run 8 took 15275ms
run 9 took 15554ms
steam_appid,coming_soon,date,appid,name,positive,negative,owners,ccu

907680,false,"Aug 17, 2018",907680,Wwbit,710,566,"100,000 .. 200,000",0

823550,false,"Sep 18, 2018",823550,Booty Calls,255,294,"100,000 .. 200,000",134

639780,false,"Dec 7, 2017",639780,Deep Space Waifu: FLAT JUSTICE,1449,67,"50,000 .. 100,000",4

steam_appid,coming_soon,date,appid,name,positive,negative,owners,ccu

896890,false,"Dec 23, 2019",896890,VR Paradise - Steam Edition,138,50,"20,000 .. 50,000",11

726360,false,"May 22, 2020",726360,BOOBS SAGA: Prepare To Hentai Edition,367,103,"20,000 .. 50,000",0

723090,false,"Oct 24, 2017",723090,Meltys Quest,446,10,"20,000 .. 50,000",36

712790,false,"Oct 2, 2017",712790,Crimson Memories,51,21,"20,000 .. 50,000",0

825300,false,"Nov 22, 2018",825300,To Trust an Incubus,43,5,"0 .. 2

# Wyniki

In [ ]:
m_s_d = " | ".join([str(x) for x in measurements_steam_df])
m_s_s = " | ".join([str(x) for x in measurements_steam_sql])
m_c_d = " | ".join([str(x) for x in measurements_covid_df])
m_c_s = " | ".join([str(x) for x in measurements_covid_sql])

print(f"|     |  covid  |  steam  |")
print(f"|-----|---------|---------|")
print(f"| df  | {sum(measurements_covid_df) / len(measurements_covid_df)} | {sum(measurements_steam_df) / len(measurements_steam_df)} |")
print(f"| sql | {sum(measurements_covid_sql) / len(measurements_covid_sql)} | {sum(measurements_steam_sql) / len(measurements_steam_sql)} |")

print()
r = " | ".join([f"run {x}" for x in range(runs)])
print(f"|         | {r} |")
print(f"|steam df |{m_s_d}|")
print(f"|steam sql|{m_s_s}|")
print(f"|covid df |{m_c_d}|")
print(f"|covid sql|{m_c_s}|")